In [ ]:
import imutils
import numpy as np
import face_recognition
import datetime
import json

# label txt => list
with open('./output_label.txt', 'r') as file:
    lines = file.readlines()
known_face_names = [line.strip() for line in lines]



# encoding txt => list
known_face_encodings = []
# 텍스트 파일을 열고 한 줄씩 읽어오기
with open('./output.txt', 'r') as file:
    lines = file.readlines()
for line in lines :
    # 줄의 시작과 끝의 대괄호를 제거하고 콤마로 분리
    line = line[1:-2]
    #print('!!!!!!!!!',line)
    l1 = []
    for number in line.split(',') :
         l1.append(number)
    known_face_encodings.append(l1)
for k in range(1770) :
    known_face_encodings[k][-1] = known_face_encodings[k][-1][:-1]
for i in range(1770) :
    for k in range(128) :
        known_face_encodings[i][k] = float(known_face_encodings[i][k])



# 결과 추출
def name_labeling(input_image):
    now = datetime.datetime.now()
    image = input_image.copy()
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image, face_locations)

    face_names = []

    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.5)
        name = "Unknown"

        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)

        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        face_names.append(name)
    
    future = datetime.datetime.now()
    print('time : ',future-now)    
    
    parsed_data = {}
    key_value_pairs = face_names[0].split(',')
    for pair in key_value_pairs:
        key, value = pair.split(':')
        parsed_data[key.strip()] = int(value) if value.strip().isdigit() else value.strip()

    # JSON 파일로 저장
    output_file_path = './output.json'
    with open(output_file_path, 'w') as json_file:
        json.dump(parsed_data, json_file, indent=4)